In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import os
import numpy as np
import torchaudio
import torch
from transformers import AutoProcessor, EncodecModel

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import os

# Set up the Google Drive authentication
gauth = GoogleAuth()
gauth.LoadCredentialsFile("mycreds.txt")
if gauth.credentials is None:
    # Authenticate if they're not there
    gauth.LocalWebserverAuth()
elif gauth.access_token_expired:
    # Refresh them if expired
    gauth.Refresh()
else:
    # Initialize the saved creds
    gauth.Authorize()
# Save the current credentials to a file
gauth.SaveCredentialsFile("mycreds.txt")

drive = GoogleDrive(gauth)

In [4]:
# Load the Encodec model and processor
processor = AutoProcessor.from_pretrained("facebook/encodec_24khz")
model = EncodecModel.from_pretrained("facebook/encodec_24khz")

# Function to process and encode a single WAV file
def encode_wav_file(file_path):
    # Load the audio file
    waveform, sample_rate = torchaudio.load(file_path)
    
    # Resample to 24kHz if necessary
    if sample_rate != 24000:
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=24000)
        waveform = resampler(waveform)

    waveform = waveform.squeeze().numpy()

    # Ensure that the audio has a single channel
    if waveform.ndim > 1:
        waveform = waveform[0]
    # Encode the waveform
    inputs = processor(waveform, sampling_rate=24000, return_tensors="pt")
    with torch.no_grad():
        audio_codes = model.encode(inputs["input_values"], inputs["padding_mask"], 24.0)
    return audio_codes[0].squeeze().cpu().numpy()

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.
C:\Users\Ripple\anaconda3\Lib\site-packages\transformers\models\encodec\modeling_encodec.py:123: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


In [5]:
# ID of the shared folder (replace with actual folder ID)
shared_folder_id = '1H8-9noFVXA-VQeSzwrOuSS8lGDLA5eB1'

# Function to list and process .wav files from a shared folder
def process_wav_files(folder_id, local_dir):
    if not os.path.exists(local_dir):
        os.makedirs(local_dir)
    file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false",
                                'includeItemsFromAllDrives': True,
                                'supportsAllDrives': True}).GetList()
    for file in file_list:
        if file['mimeType'] == 'application/vnd.google-apps.folder':
            # Recursively process .wav files in sub-folders
            process_wav_files(file['id'], os.path.join(local_dir, file['title']))
        elif file['title'].endswith('.wav'):
            file_path = os.path.join(local_dir, file['title'])
            print(f"Downloading {file['title']} to {file_path}...")
            file.GetContentFile(file_path)
            
            # Process and encode the downloaded .wav file
            try:
                audio_codes = encode_wav_file(file_path)
                audio_codes_list.append(audio_codes)
                print(f"Successfully processed {file['title']}", audio_codes.shape)
            except Exception as e:
                print(f"Failed to process {file['title']}: {e}")
            
            # Delete the file after processing
            if os.path.exists(file_path):
                os.remove(file_path)
                print(f"Deleted {file_path}")

# Local directory to save the downloaded .wav files
local_wav_dir = "temp_wav_files"

# List to store the audio codes
audio_codes_list = []

# Process the .wav files from the shared folder
process_wav_files(shared_folder_id, local_wav_dir)

# Convert the list of audio codes to a numpy array
audio_codes_array = np.array(audio_codes_list, dtype=object)

# Save the numpy array to an NPY file
np.save("audio_codes.npy", audio_codes_array)

print("All audio codes have been saved to audio_codes.npy")


Successfully processed igapo_margem_G09-2021-11-16_10-30.wav (32, 4519)
Deleted temp_wav_files\Audio\birds-of-volta-grande-do-xingu\Mïratu_09\igapo_margem_G09-2021-11-16_10-30.wav
Successfully processed igapo_margem_G09-2021-11-15_09-50.wav (32, 3844)
Deleted temp_wav_files\Audio\birds-of-volta-grande-do-xingu\Mïratu_09\igapo_margem_G09-2021-11-15_09-50.wav
Successfully processed igapo_margem_G09-2021-11-15_09-40.wav (32, 4519)
Deleted temp_wav_files\Audio\birds-of-volta-grande-do-xingu\Mïratu_09\igapo_margem_G09-2021-11-15_09-40.wav
Successfully processed igapo_margem_G09-2021-11-15_09-30.wav (32, 4519)
Deleted temp_wav_files\Audio\birds-of-volta-grande-do-xingu\Mïratu_09\igapo_margem_G09-2021-11-15_09-30.wav
Successfully processed igapo_margem_G09-2021-11-15_09-20.wav (32, 4519)
Deleted temp_wav_files\Audio\birds-of-volta-grande-do-xingu\Mïratu_09\igapo_margem_G09-2021-11-15_09-20.wav
Successfully processed igapo_margem_G09-2021-11-15_09-10.wav (32, 4519)
Deleted temp_wav_files\

KeyboardInterrupt: 